In [20]:
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.pyplot as pyplt

pyplt.rcParams["figure.figsize"] = (8, 8)

accuracy = {'bert-base-uncased': 0.5126708646905237, 'bert-large-uncased': 0.48725234219013974, 'roberta-base': 0.49738903394255873, 'roberta-large': 0.5220396252495776, 'deberta-base': 0.4888649976962064, 'deberta-large': 0.5201965903855015}


In [21]:
import pandas as pd
d = {'bert-base-uncased': 0.16691939082661103, 'bert-large-uncased': 0.2695939816828627, 'roberta-base': 0.25323999340304165, 'roberta-large': 0.29354155627190104, 'deberta-base': 0.26874974711093474, 'deberta-large': 0.2988116271661447}

for key in d:
    d[key] = round(d[key], 3)
df = pd.DataFrame(d.items(), columns=['Model', 'Macro F1-Score'])
print(df.to_latex(float_format="%.3f", index=False, column_format="@{}lll@{}"))


\begin{tabular}{@{}lll@{}}
\toprule
Model & Macro F1-Score \\
\midrule
bert-base-uncased & 0.167 \\
bert-large-uncased & 0.270 \\
roberta-base & 0.253 \\
roberta-large & 0.294 \\
deberta-base & 0.269 \\
deberta-large & 0.299 \\
\bottomrule
\end{tabular}



In [22]:
final_preds = pd.read_csv("final_preds.csv")
final_preds.drop("Unnamed: 0", axis=1, inplace=True)
import ast
# do literal_eval for each column except label and text
for col in final_preds.columns:
    if col not in ["text"]:
        final_preds[col] = final_preds[col].apply(ast.literal_eval)


In [23]:
final_preds.columns

Index(['label', 'text', 'bert-base-uncased-probs', 'bert-large-probs',
       'roberta-base-probs', 'roberta-large-probs', 'deberta-base-probs',
       'deberta-large-probs', 'bert-base-uncased-preds', 'bert-large-preds',
       'roberta-base-preds', 'roberta-large-preds', 'deberta-base-preds',
       'deberta-large-preds'],
      dtype='object')

In [24]:
pd.set_option('expand_frame_repr', False)
pd.options.display.max_colwidth = 10000
from sklearn.metrics import f1_score, accuracy_score

macro_f1 = {'bert-base-uncased': 0.16691939082661103, 
            'bert-large': 0.2695939816828627, 
            'roberta-base': 0.25323999340304165, 
            'roberta-large': 0.29354155627190104, 
            'deberta-base': 0.26874974711093474, 
            'deberta-large': 0.2988116271661447}

macro_f1_models = {'bert-base-uncased': 0.16691939082661103, 
            'bert-large': 0.2695939816828627, 
            'roberta-base': 0.25323999340304165, 
            'roberta-large': 0.29354155627190104, 
            'deberta-base': 0.26874974711093474, 
            'deberta-large': 0.2988116271661447}

f1_scores_for_classes = {
'bert-base-uncased': [0., 0.03146067, 0.21414141, 0.0776699, 0.30545455, 0.28365385, 0.22997172, 0.04545455, 0.13897281, 0.36711479, 0.03375527, 0.37660485, 0., 0., 0.14883721, 0.11604096, 0.28056112, 0.48367953, 0.03809524],
'bert-large-uncased' : [0.08450704,0.20591716,0.2539185,0.28220859,0.33764633,0.27488152,0.27322404,0.21276596,0.29580574,0.4,0.37057221,0.42592593,0.14765101,0.04347826,0.24358974,0.22666667,0.33169935,0.54939107, 0.16243655],
'roberta-base': [0.07446809, 0.19363057, 0.25505443, 0.2972973, 0.32878493, 0.2647386, 0.2546523, 0.22380952, 0.2962963, 0.35582155, 0.33333333, 0.38797423, 0.07936508, 0., 0.20869565, 0.22516556, 0.31392405, 0.525, 0.19354839],
'roberta-large': [0.10328638, 0.23529412, 0.30555556, 0.33121019, 0.36412677, 0.30861244, 0.2875817, 0.23592493, 0.27906977, 0.41947566, 0.43076923, 0.43153527, 0.14893617, 0.03636364, 0.24156306, 0.25112108, 0.34351145, 0.56779661, 0.25555556],
'deberta-base': [0.07305936, 0.20790021, 0.23876404, 0.30434783, 0.35170604, 0.2675, 0.27388535, 0.23041475, 0.31042129, 0.3810549, 0.35827664, 0.43902439, 0.16666667, 0., 0.22932331, 0.25652174, 0.32192414, 0.54545455, 0.15],
'deberta-large': [0.10714286, 0.23469388, 0.30409357, 0.35294118, 0.36971351, 0.31228473, 0.24236038, 0.26130653, 0.35897436, 0.40338983, 0.43652561, 0.42414861, 0.1732852, 0., 0.27891156, 0.27350427, 0.35959222, 0.54981084, 0.23474178]
}

for i, j in f1_scores_for_classes.items():
    print(i, len(j))

bert-base-uncased 19
bert-large-uncased 19
roberta-base 19
roberta-large 19
deberta-base 19
deberta-large 19


In [25]:
final_preds.columns
### EQUAL VOTING
## sum all -probs columns and divide by the number of columns. Then apply the threshold of 0.5 and compare with the label
d = {}
for i, row in final_preds.iterrows():
    acc_col = [0] * 19
    for col in final_preds.columns:
        if "-probs" in col:
            acc_col = [acc_col[i] + row[col][i] for i in range(19)]
    acc_col = [ 1 if x/6 > 0.2 else 0 for x in acc_col]
    d[row.text] = acc_col

final_preds['equal-voting'] = final_preds['text'].map(lambda x: list(d[x]))
macro_f1['prob-equal'] = round(f1_score(final_preds['label'].tolist(), final_preds['equal-voting'].tolist(), average='macro', zero_division=0),3)
f1_scores_for_classes['prob-equal'] = f1_score(final_preds['label'].tolist(), final_preds['equal-voting'].tolist(), average=None, zero_division=0).tolist()
accuracy['prob-equal'] = accuracy_score(final_preds['label'].tolist(), final_preds['equal-voting'].tolist())
print(macro_f1['prob-equal'])


0.331


In [26]:
final_preds.columns
## WEIGHTED VOTING. Large models vote double than base models
d = {}
for i, row in final_preds.iterrows():
    acc_col = [0] * 19
    for col in final_preds.columns:
        if "-probs" in col:
            if "base" in col:
                acc_col = [acc_col[i] + row[col][i] for i in range(19)]
            if 'large' in col:
                acc_col = [acc_col[i] + 2*row[col][i] for i in range(19)]
    acc_col = [ 1 if (x/9) > 0.3 else 0 for x in acc_col]
    d[row.text] = acc_col

final_preds['large-double-voting'] = final_preds['text'].map(lambda x: list(d[x]))
macro_f1['prob-large-double'] = round(f1_score(final_preds['label'].tolist(), final_preds['large-double-voting'].tolist(), average='macro', zero_division=0),3)
f1_scores_for_classes['prob-large-double'] = f1_score(final_preds['label'].tolist(), final_preds['large-double-voting'].tolist(), average=None, zero_division=0).tolist()
accuracy_score(final_preds['label'].tolist(), final_preds['large-double-voting'].tolist())
print(macro_f1['prob-large-double'])

0.328


In [27]:
## WEIGTHED VOTING. Models votes with their macro f1-score
d = {}

for i, row in final_preds.iterrows():
    acc_col = [0] * 19
    for col in final_preds.columns:
        if "-probs" in col:
            model = col.replace("-probs", "")
            acc_col = [acc_col[i] + macro_f1_models[model]*row[col][i] for i in range(19)]
    acc_col = [float(x / sum(macro_f1_models.values())) for x in acc_col]
    acc_col = [1 if x > 0.2 else 0 for x in acc_col]
    d[row.text] = acc_col

final_preds['f1-score-voting'] = final_preds['text'].map(lambda x: list(d[x]))
macro_f1['prob-weight-macro-f1'] = round(f1_score(final_preds['label'].tolist(), final_preds['f1-score-voting'].tolist(), average='macro', zero_division=0), 3)
f1_scores_for_classes['prob-weight-macro-f1'] = f1_score(final_preds['label'].tolist(), final_preds['f1-score-voting'].tolist(), average=None, zero_division=0).tolist()
accuracy['prob-weight-macro-f1'] = accuracy_score(final_preds['label'].tolist(), final_preds['f1-score-voting'].tolist())
print(macro_f1['prob-weight-macro-f1'])

0.334


In [28]:
## MAJORITY VOTING. On preds, the majority wins
d = {}
for i, row in final_preds.iterrows():
    acc_col = [0] * 19
    for col in final_preds.columns:
        if "-preds" in col:
            acc_col = [acc_col[i] + row[col][i] for i in range(19)]
    acc_col = [1 if x > 1 else 0 for x in acc_col]
    d[row.text] = acc_col

final_preds['majority-voting'] = final_preds['text'].map(lambda x: list(d[x]))
macro_f1['preds-majority'] = round(f1_score(final_preds['label'].tolist(), final_preds['majority-voting'].tolist(), average='macro', zero_division=0), 3)
f1_scores_for_classes['preds-majority'] = f1_score(final_preds['label'].tolist(), final_preds['majority-voting'].tolist(), average=None, zero_division=0).tolist()
accuracy['preds-majority'] = accuracy_score(final_preds['label'].tolist(), final_preds['majority-voting'].tolist())
print(macro_f1['preds-majority'])

0.323


In [29]:
## MAJORITY VOTING, DOUBLE FOR LARGE MODELS. On preds, the majority wins
d = {}
for i, row in final_preds.iterrows():
    acc_col = [0] * 19
    for col in final_preds.columns:
        if "-preds" in col:
            if "base" in col:
                acc_col = [acc_col[i] + row[col][i] for i in range(19)]
            if 'large' in col:
                acc_col = [acc_col[i] + 2*row[col][i] for i in range(19)]
    acc_col = [1 if x > 2 else 0 for x in acc_col]
    d[row.text] = acc_col

final_preds['majority-large-double-voting'] = final_preds['text'].map(lambda x: list(d[x]))
macro_f1['preds-large-double'] = round(f1_score(final_preds['label'].tolist(), final_preds['majority-large-double-voting'].tolist(), average='macro', zero_division=0), 3)
f1_scores_for_classes['preds-large-double'] = f1_score(final_preds['label'].tolist(), final_preds['majority-large-double-voting'].tolist(), average=None, zero_division=0).tolist()
accuracy['preds-large-double'] = accuracy_score(final_preds['label'].tolist(), final_preds['majority-large-double-voting'].tolist())
print(macro_f1['preds-large-double'])

0.324


In [30]:
import pandas as pd

# macro_f1['Hierocles of Alexandria $\dagger$'] = 0.39 
macro_f1['Arthur Schopenhauer $\star$ $\dagger$'] = 0.4405 
macro_f1['Philo of Alexandria'] = 0.20
df = pd.DataFrame(macro_f1.items(), columns=['Model', 'Macro F1-Score'])

print ( df.style\
.highlight_max(color='gray!25', subset=['Macro F1-Score'] ).hide(level=0, axis=0) \
.format("{:.3f}", subset=['Macro F1-Score']) \
.to_latex(convert_css=True, hrules=True, caption="Macro F1-Score Scores for the trained models. $\star$ means the model is an ensemble, and $\dagger$ means it used the multilingual dataset version",position_float='centering')
)

\begin{table}
\centering
\caption{Macro F1-Score Scores for the trained models. $\star$ means the model is an ensemble, and $\dagger$ means it used the multilingual dataset version}
\begin{tabular}{lr}
\toprule
Model & Macro F1-Score \\
\midrule
bert-base-uncased & 0.167 \\
bert-large & 0.270 \\
roberta-base & 0.253 \\
roberta-large & 0.294 \\
deberta-base & 0.269 \\
deberta-large & 0.299 \\
prob-equal & 0.331 \\
prob-large-double & 0.328 \\
prob-weight-macro-f1 & 0.334 \\
preds-majority & 0.323 \\
preds-large-double & 0.324 \\
Arthur Schopenhauer $\star$ $\dagger$ & {\cellcolor{gray!25}} 0.441 \\
Philo of Alexandria & 0.200 \\
\bottomrule
\end{tabular}
\end{table}



<>:4: SyntaxWarning: invalid escape sequence '\s'
<>:11: SyntaxWarning: invalid escape sequence '\s'
<>:4: SyntaxWarning: invalid escape sequence '\s'
<>:11: SyntaxWarning: invalid escape sequence '\s'
/var/folders/c9/g15qwq357gg9q2c0brnkt5mh0000gn/T/ipykernel_54539/513005734.py:4: SyntaxWarning: invalid escape sequence '\s'
  macro_f1['Arthur Schopenhauer $\star$ $\dagger$'] = 0.4405
/var/folders/c9/g15qwq357gg9q2c0brnkt5mh0000gn/T/ipykernel_54539/513005734.py:11: SyntaxWarning: invalid escape sequence '\s'
  .to_latex(convert_css=True, hrules=True, caption="Macro F1-Score Scores for the trained models. $\star$ means the model is an ensemble, and $\dagger$ means it used the multilingual dataset version",position_float='centering')


In [31]:
labels = ['Self-direction: thought', 'Self-direction: action', 'Stimulation', 'Hedonism', 'Achievement', 'Power: dominance', 'Power: resources', 'Face', 'Security: personal', 'Security: societal', 'Tradition', 'Conformity: rules', 'Conformity: interpersonal', 'Humility', 'Benevolence: caring', 'Benevolence: dependability', 'Universalism: concern', 'Universalism: nature', 'Universalism: tolerance']

id2label = {idx:label for idx, label in enumerate(labels)}
label2id = {label:idx for idx, label in enumerate(labels)}

In [32]:
# create a dataframe for f1_scores_for_classes using the existing dict and mapping the ids to labels
f1_scores_for_classes_df = pd.DataFrame(f1_scores_for_classes, index=[id2label[i] for i in range(19)])


In [42]:
float_format = lambda x: f"{x:.2f}".split('.')[1]

latex_table = f1_scores_for_classes_df.style \
.highlight_max(color='gray!25', axis=1) \
.format(float_format, subset=f1_scores_for_classes_df.columns)   \
.to_latex(convert_css=True, hrules=True, caption="F1-Scores for each human value",position_float='centering')

for column in f1_scores_for_classes_df.columns:
    latex_table = latex_table.replace(column, f'\\rotatebox{{90}}{{{column}}}')

print(latex_table)




\begin{table}
\centering
\caption{F1-Scores for each human value}
\begin{tabular}{lrrrrrrrrrrr}
\toprule
 & \rotatebox{90}{bert-base-uncased} & \rotatebox{90}{bert-large-uncased} & \rotatebox{90}{roberta-base} & \rotatebox{90}{roberta-large} & \rotatebox{90}{deberta-base} & \rotatebox{90}{deberta-large} & \rotatebox{90}{prob-equal} & \rotatebox{90}{prob-large-double} & \rotatebox{90}{prob-weight-macro-f1} & \rotatebox{90}{preds-majority} & \rotatebox{90}{preds-large-double} \\
\midrule
Self-direction: thought & 00 & 08 & 07 & 10 & 07 & 11 & 13 & 14 & {\cellcolor{gray!25}} 14 & 13 & 13 \\
Self-direction: action & 03 & 21 & 19 & 24 & 21 & 23 & 24 & {\cellcolor{gray!25}} 26 & 25 & 26 & 26 \\
Stimulation & 21 & 25 & 26 & 31 & 24 & 30 & {\cellcolor{gray!25}} 35 & 35 & 34 & 34 & 35 \\
Hedonism & 08 & 28 & 30 & 33 & 30 & 35 & {\cellcolor{gray!25}} 38 & 36 & 37 & 37 & 37 \\
Achievement & 31 & 34 & 33 & 36 & 35 & 37 & 40 & 40 & {\cellcolor{gray!25}} 41 & 40 & 40 \\
Power: dominance & 28 & 27 & 

In [44]:
f1_scores_for_classes_2 = {
'Hierocles of Alexandria': [ 15., 27., 30., 37., 45., 42., 49., 31., 42., 49., 46., 51., 24., 00., 34., 33., 47., 63., 27],
'Arthur Schopenhauer':     [12., 24., 33., 35., 40., 37., 47., 24., 38., 46., 49., 50., 19., 00., 32., 31., 46., 60., 27],
'Philo of Alexandria':     [ 8., 22., 27., 31., 35., 31., 34., 17., 33., 40., 47., 42., 9., 00., 21., 28., 40., 57., 21],
'prob-weight-macro-f1':    [x*100 for x in f1_scores_for_classes['prob-weight-macro-f1']],
'preds-large-double':      [x*100 for x in f1_scores_for_classes['preds-large-double']]
}

f1_general = pd.DataFrame(f1_scores_for_classes_2, index=[id2label[i] for i in range(19)])
f1_general

,Hierocles of Alexandria,Arthur Schopenhauer,Philo of Alexandria,prob-weight-macro-f1,preds-large-double
Self-direction: thought,15.0,12.0,8.0,14.022140,13.197970
Self-direction: action,27.0,24.0,22.0,24.872232,25.677267
Stimulation,30.0,33.0,27.0,34.266517,34.626039
Hedonism,37.0,35.0,31.0,37.073171,37.037037
Achievement,45.0,40.0,35.0,40.581162,39.930556
Power: dominance,42.0,37.0,31.0,35.676626,34.755463
Power: resources,49.0,47.0,34.0,33.652008,32.068164
Face,31.0,24.0,17.0,31.939163,29.879518
Security: personal,42.0,38.0,33.0,36.097561,35.814889
Security: societal,49.0,46.0,40.0,44.910891,45.480226


In [48]:
float_format = lambda x: f"{x:.02f}".split('.')[0]

latex_table = f1_general.style \
.highlight_max(color='gray!25', axis=1) \
.format(float_format, subset=f1_general.columns)   \
.to_latex(convert_css=True, hrules=True, caption="F1-Scores for each human value",position_float='centering')

for column in f1_general.columns:
    latex_table = latex_table.replace(column, f'\\rotatebox{{90}}{{{column}}}')

print(latex_table)


\begin{table}
\centering
\caption{F1-Scores for each human value}
\begin{tabular}{lrrrrr}
\toprule
 & \rotatebox{90}{Hierocles of Alexandria} & \rotatebox{90}{Arthur Schopenhauer} & \rotatebox{90}{Philo of Alexandria} & \rotatebox{90}{prob-weight-macro-f1} & \rotatebox{90}{preds-large-double} \\
\midrule
Self-direction: thought & {\cellcolor{gray!25}} 15 & 12 & 8 & 14 & 13 \\
Self-direction: action & {\cellcolor{gray!25}} 27 & 24 & 22 & 24 & 25 \\
Stimulation & 30 & 33 & 27 & 34 & {\cellcolor{gray!25}} 34 \\
Hedonism & 37 & 35 & 31 & {\cellcolor{gray!25}} 37 & 37 \\
Achievement & {\cellcolor{gray!25}} 45 & 40 & 35 & 40 & 39 \\
Power: dominance & {\cellcolor{gray!25}} 42 & 37 & 31 & 35 & 34 \\
Power: resources & {\cellcolor{gray!25}} 49 & 47 & 34 & 33 & 32 \\
Face & 31 & 24 & 17 & {\cellcolor{gray!25}} 31 & 29 \\
Security: personal & {\cellcolor{gray!25}} 42 & 38 & 33 & 36 & 35 \\
Security: societal & {\cellcolor{gray!25}} 49 & 46 & 40 & 44 & 45 \\
Tradition & 46 & {\cellcolor{gray!25}}

In [44]:
accuracy = dict(sorted(accuracy.items(), key=lambda item: item[1], reverse=True))

accuracy_df = pd.DataFrame(accuracy.items(), columns=['Model', 'Accuracy']).style \
.highlight_max(color='gray!25', subset=['Accuracy'] ).hide(level=0, axis=0) \
.format("{:.3f}", subset=['Accuracy'])   \
.to_latex(convert_css=True, hrules=True, caption="Accuracy Scores for the trained models",position_float='centering')
print(accuracy_df)

\begin{table}
\centering
\caption{Accuracy Scores for the trained models}
\begin{tabular}{lr}
\toprule
Model & Accuracy \\
\midrule
roberta-large & {\cellcolor{gray!25}} 0.522 \\
deberta-large & 0.520 \\
bert-base-uncased & 0.513 \\
preds-large-double & 0.503 \\
roberta-base & 0.497 \\
preds-majority & 0.493 \\
deberta-base & 0.489 \\
bert-large-uncased & 0.487 \\
prob-equal & 0.452 \\
prob-weight-macro-f1 & 0.451 \\
\bottomrule
\end{tabular}
\end{table}

